# Modelo 2030 - Conversión de script a Notebook
Este notebook contiene la conversión del script `scene_2030.py` a celdas Jupyter organizadas: imports, carga de datos, parámetros, definición del modelo Pyomo, resolución y resultados.

Asegúrate de tener instaladas las dependencias: `pandas`, `pyomo`, y el solver `HiGHS` (o ajustar el solver si no está disponible).

## 1. Importar Librerías y Configuración Inicial

In [2]:
# Imports
import pandas as pd
import highspy as hgs
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Parámetros de configuración
tolerancia = 0.00001
perdida = 0.04

## 2. Cargar y Procesar Datos de Centrales Eléctricas

In [3]:
# Cargar archivos CSV (asegúrate que los archivos existan en el mismo directorio)
df_centrales_ex = pd.read_csv('centrales_ex_ED_CS.csv')
df_centrales_ex = df_centrales_ex.fillna(0)

df_centrales_nuevas = pd.read_csv('centrales_n_ED_CS.csv')
df_centrales_nuevas = df_centrales_nuevas.fillna(0)

# Preparar parámetros (index por nombre de planta)
param_centrales = df_centrales_ex.set_index(['planta_n'])
param_centrales_nuevas = df_centrales_nuevas.set_index(['planta_n'])

# Revisar las primeras filas
#display(df_centrales_ex.head())
#display(df_centrales_nuevas.head())

# 3. Carga de Datos de Equipo de Abatidores y Daño Ambiental

In [4]:
# Equipos de abatimiento
df_equipo_mp = pd.read_csv('abatimiento/equipo_mp.csv')
df_equipo_mp = df_equipo_mp.fillna(0)
equipo_mp = df_equipo_mp.set_index(['Equipo_MP'])

df_equipo_nox = pd.read_csv('abatimiento/equipo_nox.csv')
df_equipo_nox = df_equipo_nox.fillna(0)
equipo_nox = df_equipo_nox.set_index(['Equipo_NOx'])

df_equipo_sox = pd.read_csv('abatimiento/equipo_sox.csv')
df_equipo_sox = df_equipo_sox.fillna(0)
equipo_sox = df_equipo_sox.set_index(['Equipo_SOx'])

# Daño ambiental (costo social)

df_dano_ambiental = pd.read_csv('dano_ambiental.csv')
df_dano_ambiental = df_dano_ambiental.fillna(0)
dano_ambiental = df_dano_ambiental.set_index(['Ubicacion'])

display(equipo_mp.head())
display(equipo_nox.head())
display(equipo_sox.head())
display(dano_ambiental.head())

,Eficiencia_(p.u.),Inversión_($/kW),Costo_variable_($/MWh)
Equipo_MP,,,
MP-1,0.90,116,1.5
MP-2,0.98,158,2.0
MP-3,0.99,274,2.0


,Eficiencia_(p.u.),Inversión_($/kW),Costo_variable_($/MWh)
Equipo_NOx,,,
NOx-1,0.70,41,1.5
NOx-2,0.90,42,2.0
NOx-3,0.95,63,2.0


,Eficiencia_(p.u.),Inversión_($/kW),Costo_variable_($/MWh)
Equipo_SOx,,,
SOx-1,0.60,211,1.5
SOx-2,0.90,263,2.0
SOx-3,0.95,316,2.0


,MP_($/ton),SOx_($/ton),NOx_($/ton),CO2_($/ton)
Ubicacion,,,,
Diego de Almagro,527,211,105,50
Cardones,527,211,105,50
Huasco,527,211,105,50
Pan de Azúcar,527,211,105,50
Los Vilos,3161,1581,2107,50


## 4. Definir Parámetros del Sistema y Constantes

In [5]:
# Tipos de Centrales
t_centrales = ['biomasa', 'carbon','cc-gnl', 'petroleo_diesel', 'hidro', 'minihidro','eolica','solar', 'geotermia']
t_ernc = ['eolica','solar', 'geotermia','minihidro','hidro' ]
t_centrales_termicas = ['carbon','cc-gnl', 'petroleo_diesel']
dispnibilidad_hidro = [0.8215,0.6297,0.561]
costo_falla = 505.5
year = 2030 - 2016

# Constantes para pasar de GWh a Toneladas de combustible (calculado en el excel)
conversion_GWh_a_ton = {'petroleo_diesel': 79.6808152,
                         'cc-gnl' : 61.96031117,
                            'carbon': 132.3056959}

## Se supone que si cada uno de estos factores lo multiplicamos GWh * factor * emisiones descontroladas
## nos da las emisiones en kg de cada contaminante (MP, SOx, NOx, CO2) 
## luego hay que pasarlo a toneladas dividiendo por 1000
## y finalmente para pasarlo a costo social hay que multiplicarlo por el costo social de cada contaminante


## 5. Definir Índices de Plantas y Bloques de Carga

In [6]:
# Índices Centrales
index_plantas = param_centrales.index.tolist()
index_plantas_nuevas = param_centrales_nuevas.index.tolist()  # demanda en MW (en la pasada tenia 12k en vez de 1200 xd)

# Índices Equipos de Abatimiento
index_equipo_mp = equipo_mp.index.tolist() # MP-1, MP-2, MP-3
index_equipo_nox = equipo_nox.index.tolist() # NOx-1, NOx-2, NOx-3
index_equipo_sox = equipo_sox.index.tolist() # SOx-1, SOx-2, SOx-3

dic_bloques = {'bloque_1': {'duracion': 1200 , 'demanda' : 10233.87729},
               'bloque_2': {'duracion': 4152 , 'demanda' : 7872.0103}, 
               'bloque_3': {'duracion': 3408 , 'demanda' : 6297.872136}}

normas_emision = {'carbon': {'MP': 0.99 , 'SOx' : 0.95, 'NOx' : 0.0},
               'cc-gnl': {'MP': 0.95 , 'SOx' : 0.0, 'NOx' : 0.9}, 
               'petroleo_diesel': {'MP': 0.95 , 'SOx' : 0.0, 'NOx' : 0.0}}

vida_util = 30 # todos los equipos tienen vida util de 30 años 

# Mostrar resumen
print('Plantillas existentes:', index_plantas[:5])
print('Plantillas nuevas:', index_plantas_nuevas[:5])

Plantillas existentes: ['planta_1', 'planta_2', 'planta_3', 'planta_4', 'planta_5']
Plantillas nuevas: ['planta_1', 'planta_2', 'planta_3', 'planta_4', 'planta_5']


In [7]:
dic = equipo_mp.to_dict()
print(dic)

{'Eficiencia_(p.u.)': {'MP-1': 0.9, 'MP-2': 0.98, 'MP-3': 0.99}, 'Inversión_($/kW)': {'MP-1': 116, 'MP-2': 158, 'MP-3': 274}, 'Costo_variable_($/MWh)': {'MP-1': 1.5, 'MP-2': 2.0, 'MP-3': 2.0}}


In [29]:
dic_equipo = {'MP':equipo_mp.to_dict(orient='index'), 
              'NOx':equipo_nox.to_dict(orient='index'), 
              'SOx':equipo_sox.to_dict(orient='index')}
print(dic_equipo)

#print escalonado para mostrar  lo que hay en cada diccionario
for key in dic_equipo:
    print(f"Equipos para {key}:")
    for equipo, valores in dic_equipo[key].items():
        print(f"  {equipo}: {valores}")
        


{'MP': {'MP-1': {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 116, 'Costo_variable_($/MWh)': 1.5}, 'MP-2': {'Eficiencia_(p.u.)': 0.98, 'Inversión_($/kW)': 158, 'Costo_variable_($/MWh)': 2.0}, 'MP-3': {'Eficiencia_(p.u.)': 0.99, 'Inversión_($/kW)': 274, 'Costo_variable_($/MWh)': 2.0}}, 'NOx': {'NOx-1': {'Eficiencia_(p.u.)': 0.7, 'Inversión_($/kW)': 41, 'Costo_variable_($/MWh)': 1.5}, 'NOx-2': {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 42, 'Costo_variable_($/MWh)': 2.0}, 'NOx-3': {'Eficiencia_(p.u.)': 0.95, 'Inversión_($/kW)': 63, 'Costo_variable_($/MWh)': 2.0}}, 'SOx': {'SOx-1': {'Eficiencia_(p.u.)': 0.6, 'Inversión_($/kW)': 211, 'Costo_variable_($/MWh)': 1.5}, 'SOx-2': {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 263, 'Costo_variable_($/MWh)': 2.0}, 'SOx-3': {'Eficiencia_(p.u.)': 0.95, 'Inversión_($/kW)': 316, 'Costo_variable_($/MWh)': 2.0}}}
Equipos para MP:
  MP-1: {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 116, 'Costo_variable_($/MWh)': 1.5}
  MP-2: {'Eficiencia_(p.u.)': 

## 6. Crear Modelo y Definir Conjuntos | Parámetros | Variables

In [ ]:
# Construcción del modelo Pyomo
model = pyo.ConcreteModel()

# Conjuntos
model.CENTRALES = pyo.Set(initialize=index_plantas)
model.CENTRALES_NUEVAS = pyo.Set(initialize=index_plantas_nuevas)
model.BLOQUES = pyo.Set(initialize=['bloque_1','bloque_2','bloque_3'])
model.ABATIMIENTO_MP = pyo.Set(initialize=['MP-1','MP-2','MP-3']) 
model.ABATIMIENTO_NOX = pyo.Set(initialize=['NOx-1','NOx-2','NOx-3']) 
model.ABATIMIENTO_SOX = pyo.Set(initialize=['SOx-1','SOx-2','SOx-3'])

# Parámetros en formato dict (Pyomo Any para permitir dicts anidados)
model.param_centrales = pyo.Param(model.CENTRALES, initialize=param_centrales.to_dict(orient='index'), within=pyo.Any)
model.param_centrales_nuevas = pyo.Param(model.CENTRALES_NUEVAS, initialize=param_centrales_nuevas.to_dict(orient='index'), within=pyo.Any)
model.param_bloques = pyo.Param(model.BLOQUES, initialize=dic_bloques, within=pyo.Any)
model.param_abatimiento_mp = pyo.Param(model.ABATIMIENTO_MP, initialize=equipo_mp.to_dict(orient='index'), within=pyo.Any)
model.param_abatimiento_nox = pyo.Param(model.ABATIMIENTO_NOX, initialize=equipo_nox.to_dict(orient='index'), within=pyo.Any)
model.param_abatimiento_sox = pyo.Param(model.ABATIMIENTO_SOX, initialize=equipo_sox.to_dict(orient='index'), within=pyo.Any)

# command and control equipment parameters


# Variables(todas las generaciones son en GWh y la potencia en MW)
model.generacion_ex = pyo.Var(model.CENTRALES, model.BLOQUES, within=pyo.NonNegativeReals)
model.generacion_nuevas = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, within=pyo.NonNegativeReals)
model.potencia_in_nuevas = pyo.Var(model.CENTRALES_NUEVAS, within=pyo.NonNegativeReals)
model.falla = pyo.Var(model.BLOQUES, within=pyo.NonNegativeReals)

model.abatimiento_mp_ex = pyo.Var(model.CENTRALES, model.ABATIMIENTO_MP, within=pyo.Binary)
model.abatimiento_nox_ex = pyo.Var(model.CENTRALES, model.ABATIMIENTO_NOX, within=pyo.Binary)
model.abatimiento_sox_ex = pyo.Var(model.CENTRALES, model.ABATIMIENTO_SOX, within=pyo.Binary)

model.abatimiento_mp_new = pyo.Var(model.CENTRALES_NUEVAS, model.ABATIMIENTO_MP, within=pyo.Binary)
model.abatimiento_nox_new = pyo.Var(model.CENTRALES_NUEVAS, model.ABATIMIENTO_NOX, within=pyo.Binary)
model.abatimiento_sox_new = pyo.Var(model.CENTRALES_NUEVAS, model.ABATIMIENTO_SOX, within=pyo.Binary)   

Big_M = 1e5  # Una gran constante para las restricciones de selección de tecnología

# variables binarias que dicen si generamos o no
model.generacion_bin_ex = pyo.Var(model.CENTRALES, model.BLOQUES, within=pyo.Binary)
model.generacion_bin_new = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, within=pyo.Binary)



In [15]:
print(model.param_abatimiento_mp["MP-1"])

{'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 116, 'Costo_variable_($/MWh)': 1.5}


## 7. Definir Funciones de Restricciones

In [16]:
# Restricciones
def fd_hidro(bloque):
    if bloque == 'bloque_1':
        return dispnibilidad_hidro[0]
    elif bloque == 'bloque_2':
        return dispnibilidad_hidro[1]
    else:
        return dispnibilidad_hidro[2]

def balance_demanda(model, bloque):
    gen_ex = sum(model.generacion_ex[planta, bloque] for planta in model.CENTRALES)
    gen_new = sum(model.generacion_nuevas[planta, bloque] for planta in model.CENTRALES_NUEVAS)
    return (gen_ex + gen_new + model.falla[bloque]) * (1000/(1+perdida)) >= model.param_bloques[bloque]['demanda'] * model.param_bloques[bloque]['duracion']
                                                    # el 1000 es para convertir de GWh a MWh

def max_gen_ex(model, planta, bloque):
    tec = model.param_centrales[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'hidro_conv', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales[planta]['disponibilidad']
        efi = model.param_centrales[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_ex[planta, bloque]*1000 <= model.param_centrales[planta]['potencia_neta_mw'] * model.param_bloques[bloque]['duracion'] * disp

def max_gen_nuevas(model, planta, bloque):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales_nuevas[planta]['disponibilidad']
        efi = model.param_centrales_nuevas[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_nuevas[planta, bloque]*1000 <= model.potencia_in_nuevas[planta] * model.param_bloques[bloque]['duracion'] * disp

def max_capacidad_nuevas(model, planta):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_ernc:
        limite = model.param_centrales_nuevas[planta]['maxima_restriccion_2030_MW']
        return model.potencia_in_nuevas[planta] <= limite if limite > 0 else pyo.Constraint.Skip # Si el límite es 0, no hay restricción
    else:
        return pyo.Constraint.Skip
    
def solo_uno_mp_new(model, planta):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_centrales_termicas: 
        # esto es para las centrales termicas
        return sum(model.abatimiento_mp_new[planta, equipo] for equipo in model.ABATIMIENTO_MP) <= 1
    else:
        # si no es termica, no aplica restricción (se salta)
        return pyo.Constraint.Skip

def solo_uno_nox_new(model, planta):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_centrales_termicas: 
        # esto es para las centrales termicas
        return sum(model.abatimiento_nox_new[planta, equipo] for equipo in model.ABATIMIENTO_NOX) <= 1
    else:
        # si no es termica, no aplica restricción (se salta)
        return pyo.Constraint.Skip
    
def solo_uno_sox_new(model, planta):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_centrales_termicas: 
        # esto es para las centrales termicas
        return sum(model.abatimiento_sox_new[planta, equipo] for equipo in model.ABATIMIENTO_SOX) <= 1
    else:
        # si no es termica, no aplica restricción (se salta)
        return pyo.Constraint.Skip

def solo_uno_mp_ex(model, planta):
    tec = model.param_centrales[planta]['tecnologia']
    if tec in t_centrales_termicas: 
        # esto es para las centrales termicas
        return sum(model.abatimiento_mp_ex[planta, equipo] for equipo in model.ABATIMIENTO_MP) <= 1
    else:
        # si no es termica, no aplica restricción (se salta)
        return pyo.Constraint.Skip
    
def solo_uno_nox_ex(model, planta):
    tec = model.param_centrales[planta]['tecnologia']
    if tec in t_centrales_termicas: 
        # esto es para las centrales termicas
        return sum(model.abatimiento_nox_ex[planta, equipo] for equipo in model.ABATIMIENTO_NOX) <= 1
    else:
        # si no es termica, no aplica restricción (se salta)
        return pyo.Constraint.Skip

def solo_uno_sox_ex(model, planta):
    tec = model.param_centrales[planta]['tecnologia']
    if tec in t_centrales_termicas: 
        # esto es para las centrales termicas
        return sum(model.abatimiento_sox_ex[planta, equipo] for equipo in model.ABATIMIENTO_SOX) <= 1
    else:
        # si no es termica, no aplica restricción (se salta)
        return pyo.Constraint.Skip


def norma_emision_existente(planta):
    ed_mp = model.param_centrales[planta]['emisiones_mp_kg_mwh']
    ed_sox = model.param_centrales[planta]['emisiones_sox_kg_mwh']
    ed_nox = model.param_centrales[planta]['emisiones_nox_kg_mwh']
    tec = model.param_centrales[planta]['tecnologia']
    if tec in t_centrales_termicas:
        if tec == 'carbon': # mp y sox
            mp_sin_abat = sum(model.generacion_ex[planta, bloque] * ed_mp * conversion_GWh_a_ton["carbon"] for bloque in model.BLOQUES)
            sox_sin_abat = sum(model.generacion_ex[planta, bloque] * ed_sox * conversion_GWh_a_ton["carbon"] for bloque in model.BLOQUES)

            mp_con_abat = mp_sin_abat * (1 - sum(model.abatimiento_mp_ex[planta, equipo] * model.param_abatimiento_mp[equipo]['eficiencia'] 
                                            for equipo in model.ABATIMIENTO_MP))
            sox_con_abat = sox_sin_abat * (1 - sum(model.abatimiento_sox_ex[planta, equipo] * model.param_abatimiento_sox[equipo]['eficiencia'] 
                                            for equipo in model.ABATIMIENTO_SOX))

            pass
        elif tec == 'cc-gnl': # mp y nox
            pass
        elif tec == 'petroleo_diesel': # mp
            pass
    else:
        # si no es termica, no aplica restricción (se salta)
        return pyo.Constraint.Skip


def anualidad(r, n): # r es tasa de descuento, n es vida util en años
    return r / (1 - (1 + r)**(-n))

# Adjuntar restricciones
model.demanda_constraint = pyo.Constraint(model.BLOQUES, rule=balance_demanda)
model.max_gen_constraint = pyo.Constraint(model.CENTRALES, model.BLOQUES, rule=max_gen_ex)
model.max_gen_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, rule=max_gen_nuevas)
model.max_capacidad_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, rule=max_capacidad_nuevas)

# Agregar constraints de "solo uno" para abatidores
model.solo_uno_mp_new_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, rule=solo_uno_mp_new)
model.solo_uno_nox_new_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, rule=solo_uno_nox_new)
model.solo_uno_sox_new_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, rule=solo_uno_sox_new)

model.solo_uno_mp_ex_constraint = pyo.Constraint(model.CENTRALES, rule=solo_uno_mp_ex)
model.solo_uno_nox_ex_constraint = pyo.Constraint(model.CENTRALES, rule=solo_uno_nox_ex)
model.solo_uno_sox_ex_constraint = pyo.Constraint(model.CENTRALES, rule=solo_uno_sox_ex)


IndentationError: expected an indented block after function definition on line 101 (2156961717.py, line 103)

## 9. Definir Expresiones de Costos y Función Objetivo

In [110]:
# Función objetivo (expresiones)

# Costo operación (costos variables) 2030
model.op_ex = pyo.Expression(expr=sum(model.generacion_ex[planta, bloque] * 1000 * # pasamos a MWh
                                (model.param_centrales[planta]['costo_variable_nc']
                                + model.param_centrales[planta]['costo_variable_t']
                                + model.param_centrales[planta]['costo_var_comb_16_usd_mwh']) 
                                for planta in model.CENTRALES 
                                for bloque in model.BLOQUES)) # esto queda en USD

model.op_new = pyo.Expression(expr=sum(model.generacion_nuevas[planta, bloque] * 1000 * # pasamos a MWh
                                (model.param_centrales_nuevas[planta]['cvnc_usd_MWh'] 
                                + model.param_centrales_nuevas[planta]['linea_peaje_usd_MWh']
                                + model.param_centrales_nuevas[planta]['costo_var_comb_16_usd_mwh'])  
                                for planta in model.CENTRALES_NUEVAS 
                                for bloque in model.BLOQUES)) # esto queda en USD

# Costo inversión (anualidad) 2030
model.inv_new = pyo.Expression(expr=sum(model.potencia_in_nuevas[planta]* 
                                        anualidad(model.param_centrales_nuevas[planta]['tasa_descuento'], model.param_centrales_nuevas[planta]['vida_util_anos']) * 
                                        model.param_centrales_nuevas[planta]['inversion_usd_kW_neto']* 1000 # pasamos a MW
                                        for planta in model.CENTRALES_NUEVAS)) # esto queda en USD/año

# costo FALLAS (recordar que falla está en GWh)
model.costo_fallas = pyo.Expression(expr=sum(model.falla[bloque] * 1000 * # pasamos a MWh
                            costo_falla for bloque in model.BLOQUES))

r_df = 0.01
df_2016_2030 = 1 / (1 + r_df)**(year)

# minimizar (costo operacion + inversion + costo falla)
model.obj = pyo.Objective(expr = df_2016_2030 * (model.op_ex + model.op_new + model.inv_new + model.costo_fallas), sense = pyo.minimize)

## 10. Resolver el Modelo de Optimización

In [111]:
# Resolver el modelo
solver = pyo.SolverFactory('highs')
solver.options['mip_rel_gap'] = tolerancia
results = solver.solve(model, tee=True)
print(f"Status: {results}")

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 131 rows; 146 cols; 314 nonzeros
Coefficient ranges:
  Matrix [1e+00, 4e+03]
  Cost   [9e+03, 4e+05]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 3e+07]
Presolving model
63 rows, 143 cols, 243 nonzeros  0s
63 rows, 140 cols, 240 nonzeros  0s
Presolve : Reductions: rows 63(-68); columns 140(-6); elements 240(-74)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(4.3901e+07) 0s
         40     1.5338548176e+09 Pr: 0(0); Du: 0(1.42109e-14) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 40
Objective value     :  1.5338548176e+09
P-D objective error :  1.5543751357e-16
HiGHS run time      :          0.00
Status: 
Problem: 
- Lower bound: 1533854817.584971
  Upper bound: 1533854817.584971
  Number of objectives: 1
  Number 

## 11. Análisis de Resultados por Tipo de Tecnología

In [112]:
# ver resultados por tipo de central
for tec in t_centrales:
    gen_tec = sum(model.generacion_ex[planta, bloque].value 
                  for planta in model.CENTRALES if model.param_centrales[planta]['tecnologia'] == tec
                  for bloque in model.BLOQUES)
    gen_tec_nuevas = sum(model.generacion_nuevas[planta, bloque].value 
                  for planta in model.CENTRALES_NUEVAS if model.param_centrales_nuevas[planta]['tecnologia'] == tec
                  for bloque in model.BLOQUES)
    gen_tec += gen_tec_nuevas
    print(f'Generación total de tecnología {tec}: {gen_tec} GWh')

Generación total de tecnología biomasa: 1271.6352000000002 GWh
Generación total de tecnología carbon: 34581.53984929151 GWh
Generación total de tecnología cc-gnl: 815.796 GWh
Generación total de tecnología petroleo_diesel: 849.5926435200009 GWh
Generación total de tecnología hidro: 30708.4795704 GWh
Generación total de tecnología minihidro: 0.0 GWh
Generación total de tecnología eolica: 420.48 GWh
Generación total de tecnología solar: 438.0 GWh
Generación total de tecnología geotermia: 0.0 GWh


## 12. Análisis de Generación Total del Sistema y Balance de Carga

In [113]:
#sumatoria todas las generaciones
total = 0
for bloque in model.BLOQUES:
    gen_ex_bloque = sum(model.generacion_ex[planta, bloque].value for planta in model.CENTRALES)
    gen_new_bloque = sum(model.generacion_nuevas[planta, bloque].value for planta in model.CENTRALES_NUEVAS)
    falla_bloque = model.falla[bloque].value
    print(f'Generación total en {bloque}: {gen_ex_bloque+gen_new_bloque} GWh, Falla: {falla_bloque} GWh')
    total += gen_ex_bloque + gen_new_bloque + falla_bloque

print(f'Generación total en el sistema (incluyendo fallas): {total} GWh')

Generación total en bloque_1: 12771.87885792 GWh, Falla: 0.0 GWh
Generación total en bloque_2: 33991.970236224 GWh, Falla: 0.0 GWh
Generación total en bloque_3: 22321.67416906752 GWh, Falla: 0.0 GWh
Generación total en el sistema (incluyendo fallas): 69085.52326321151 GWh


In [ ]:
# --- Emisiones: linearización, variables auxiliares y activación por cumplimiento de norma ---
# Calculamos BigM en GWh a partir de la potencia máxima y la duración máxima de bloques
max_pot_mw = 0
for p in index_plantas + index_plantas_nuevas:
    try:
        pot = float(param_centrales.loc[p, 'potencia_neta_mw']) if p in param_centrales.index else float(param_centrales_nuevas.loc[p, 'potencia_neta_mw'])
    except Exception:
        pot = 0.0
    if pot > max_pot_mw:
        max_pot_mw = pot

max_duracion_h = max([model.param_bloques[b]['duracion'] for b in model.BLOQUES])
BigM_GWh = (max_pot_mw * max_duracion_h) / 1000.0  # MW * h -> MWh; /1000 -> GWh

# Estimación BigM para emisiones en kg: máxima generación (GWh)*1000*MÁX_ED(kg/MWh)
max_ed = 0.0
for p in index_plantas + index_plantas_nuevas:
    if p in param_centrales.index:
        for col in ['emisiones_mp_kg_mwh', 'emisiones_sox_kg_mwh', 'emisiones_nox_kg_mwh']:
            try:
                val = float(param_centrales.loc[p, col])
            except Exception:
                val = 0.0
            if val > max_ed:
                max_ed = val
    elif p in param_centrales_nuevas.index:
        for col in ['emisiones_mp_kg_mwh', 'emisiones_sox_kg_mwh', 'emisiones_nox_kg_mwh']:
            try:
                val = float(param_centrales_nuevas.loc[p, col])
            except Exception:
                val = 0.0
            if val > max_ed:
                max_ed = val

BigM_emis_kg = BigM_GWh * 1000.0 * max_ed * len(list(model.BLOQUES))

# Variables auxiliares para linealizar gen * seleccion de equipo
model.y_mp_ex = pyo.Var(model.CENTRALES, model.BLOQUES, model.ABATIMIENTO_MP, within=pyo.NonNegativeReals)
model.y_sox_ex = pyo.Var(model.CENTRALES, model.BLOQUES, model.ABATAMIENTO_SOX, within=pyo.NonNegativeReals)
model.y_nox_ex = pyo.Var(model.CENTRALES, model.BLOQUES, model.ABATIMIENTO_NOX, within=pyo.NonNegativeReals)

model.y_mp_new = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATIMIENTO_MP, within=pyo.NonNegativeReals)
model.y_sox_new = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATAMIENTO_SOX, within=pyo.NonNegativeReals)
model.y_nox_new = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATIMIENTO_NOX, within=pyo.NonNegativeReals)

# Restricciones de linealización para existentes (y <= gen; y <= BigM*select; y >= gen - BigM*(1-select))
def y_le_gen_rule(m, planta, bloque, equipo):
    return m.y_mp_ex[planta, bloque, equipo] <= m.generacion_ex[planta, bloque]

def y_le_bigM_sel_rule(m, planta, bloque, equipo):
    return m.y_mp_ex[planta, bloque, equipo] <= BigM_GWh * m.abatimiento_mp_ex[planta, equipo]

def y_ge_gen_minus_bigM_rule(m, planta, bloque, equipo):
    return m.y_mp_ex[planta, bloque, equipo] >= m.generacion_ex[planta, bloque] - BigM_GWh * (1 - m.abatimiento_mp_ex[planta, equipo])

model.y_mp_le_gen = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATIMIENTO_MP, rule=y_le_gen_rule)
model.y_mp_le_bigM_sel = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATIMIENTO_MP, rule=y_le_bigM_sel_rule)
model.y_mp_ge_gen_minus_bigM = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATIMIENTO_MP, rule=y_ge_gen_minus_bigM_rule)

# Repetir para NOx
def y_nox_le_gen_rule(m, planta, bloque, equipo):
    return m.y_nox_ex[planta, bloque, equipo] <= m.generacion_ex[planta, bloque]

def y_nox_le_bigM_sel_rule(m, planta, bloque, equipo):
    return m.y_nox_ex[planta, bloque, equipo] <= BigM_GWh * m.abatimiento_nox_ex[planta, equipo]

def y_nox_ge_gen_minus_bigM_rule(m, planta, bloque, equipo):
    return m.y_nox_ex[planta, bloque, equipo] >= m.generacion_ex[planta, bloque] - BigM_GWh * (1 - m.abatimiento_nox_ex[planta, equipo])

model.y_nox_le_gen = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATIMIENTO_NOX, rule=y_nox_le_gen_rule)
model.y_nox_le_bigM_sel = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATAMIENTO_NOX, rule=y_nox_le_bigM_sel_rule)
model.y_nox_ge_gen_minus_bigM = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATAMIENTO_NOX, rule=y_nox_ge_gen_minus_bigM_rule)

# Repetir para SOx
def y_sox_le_gen_rule(m, planta, bloque, equipo):
    return m.y_sox_ex[planta, bloque, equipo] <= m.generacion_ex[planta, bloque]

def y_sox_le_bigM_sel_rule(m, planta, bloque, equipo):
    return m.y_sox_ex[planta, bloque, equipo] <= BigM_GWh * m.abatimiento_sox_ex[planta, equipo]

def y_sox_ge_gen_minus_bigM_rule(m, planta, bloque, equipo):
    return m.y_sox_ex[planta, bloque, equipo] >= m.generacion_ex[planta, bloque] - BigM_GWh * (1 - m.abatimiento_sox_ex[planta, equipo])

model.y_sox_le_gen = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATIMIENTO_SOX, rule=y_sox_le_gen_rule)
model.y_sox_le_bigM_sel = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATAMIENTO_SOX, rule=y_sox_le_bigM_sel_rule)
model.y_sox_ge_gen_minus_bigM = pyo.Constraint(model.CENTRALES, model.BLOQUES, model.ABATAMIENTO_SOX, rule=y_sox_ge_gen_minus_bigM_rule)

# --- Repetir linealización para plantas nuevas ---
# MP
def y_mp_new_le_gen_rule(m, planta, bloque, equipo):
    return m.y_mp_new[planta, bloque, equipo] <= m.generacion_nuevas[planta, bloque]

def y_mp_new_le_bigM_sel_rule(m, planta, bloque, equipo):
    return m.y_mp_new[planta, bloque, equipo] <= BigM_GWh * m.abatimiento_mp_new[planta, equipo]

def y_mp_new_ge_gen_minus_bigM_rule(m, planta, bloque, equipo):
    return m.y_mp_new[planta, bloque, equipo] >= m.generacion_nuevas[planta, bloque] - BigM_GWh * (1 - m.abatimiento_mp_new[planta, equipo])

model.y_mp_new_le_gen = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATIMIENTO_MP, rule=y_mp_new_le_gen_rule)
model.y_mp_new_le_bigM_sel = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATIMIENTO_MP, rule=y_mp_new_le_bigM_sel_rule)
model.y_mp_new_ge_gen_minus_bigM = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATIMIENTO_MP, rule=y_mp_new_ge_gen_minus_bigM_rule)

# NOx
def y_nox_new_le_gen_rule(m, planta, bloque, equipo):
    return m.y_nox_new[planta, bloque, equipo] <= m.generacion_nuevas[planta, bloque]

def y_nox_new_le_bigM_sel_rule(m, planta, bloque, equipo):
    return m.y_nox_new[planta, bloque, equipo] <= BigM_GWh * m.abatimiento_nox_new[planta, equipo]

def y_nox_new_ge_gen_minus_bigM_rule(m, planta, bloque, equipo):
    return m.y_nox_new[planta, bloque, equipo] >= m.generacion_nuevas[planta, bloque] - BigM_GWh * (1 - m.abatimiento_nox_new[planta, equipo])

model.y_nox_new_le_gen = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATIMIENTO_NOX, rule=y_nox_new_le_gen_rule)
model.y_nox_new_le_bigM_sel = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATAMIENTO_NOX, rule=y_nox_new_le_bigM_sel_rule)
model.y_nox_new_ge_gen_minus_bigM = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATAMIENTO_NOX, rule=y_nox_new_ge_gen_minus_bigM_rule)

# SOx
def y_sox_new_le_gen_rule(m, planta, bloque, equipo):
    return m.y_sox_new[planta, bloque, equipo] <= m.generacion_nuevas[planta, bloque]

def y_sox_new_le_bigM_sel_rule(m, planta, bloque, equipo):
    return m.y_sox_new[planta, bloque, equipo] <= BigM_GWh * m.abatimiento_sox_new[planta, equipo]

def y_sox_new_ge_gen_minus_bigM_rule(m, planta, bloque, equipo):
    return m.y_sox_new[planta, bloque, equipo] >= m.generacion_nuevas[planta, bloque] - BigM_GWh * (1 - m.abatimiento_sox_new[planta, equipo])

model.y_sox_new_le_gen = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATIMIENTO_SOX, rule=y_sox_new_le_gen_rule)
model.y_sox_new_le_bigM_sel = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATAMIENTO_SOX, rule=y_sox_new_le_bigM_sel_rule)
model.y_sox_new_ge_gen_minus_bigM = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, model.ABATAMIENTO_SOX, rule=y_sox_new_ge_gen_minus_bigM_rule)

# Variables binarias por planta que indican si la planta está permitida a generar (cumple norma)
model.planta_operable_ex = pyo.Var(model.CENTRALES, within=pyo.Binary)
model.planta_operable_new = pyo.Var(model.CENTRALES_NUEVAS, within=pyo.Binary)

# Forzar que si planta_operable == 0 no haya generación en ningún bloque (y los binaries de bloque también sean 0)
def gen_allowed_existing_rule(m, planta, bloque):
    return m.generacion_ex[planta, bloque] <= BigM_GWh * m.planta_operable_ex[planta]

def gen_bin_allowed_existing_rule(m, planta, bloque):
    return m.generacion_bin_ex[planta, bloque] <= m.planta_operable_ex[planta]

model.gen_allowed_existing = pyo.Constraint(model.CENTRALES, model.BLOQUES, rule=gen_allowed_existing_rule)
model.gen_bin_allowed_existing = pyo.Constraint(model.CENTRALES, model.BLOQUES, rule=gen_bin_allowed_existing_rule)

# Repetir para nuevas
def gen_allowed_new_rule(m, planta, bloque):
    return m.generacion_nuevas[planta, bloque] <= BigM_GWh * m.planta_operable_new[planta]

def gen_bin_allowed_new_rule(m, planta, bloque):
    return m.generacion_bin_new[planta, bloque] <= m.planta_operable_new[planta]

model.gen_allowed_new = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, rule=gen_allowed_new_rule)
model.gen_bin_allowed_new = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, rule=gen_bin_allowed_new_rule)

# Expresiones de emisiones sin abatimiento, abatidas y remanentes (kg) por planta

def emis_sin_kg_existing(m, planta, contaminante):
    # contaminante in {'MP','SOx','NOx'}
    if contaminante == 'MP':
        col = 'emisiones_mp_kg_mwh'
    elif contaminante == 'SOx':
        col = 'emisiones_sox_kg_mwh'
    else:
        col = 'emisiones_nox_kg_mwh'
    ed = float(m.param_centrales[planta][col])
    return sum(m.generacion_ex[planta, bloque] * 1000.0 * ed for bloque in m.BLOQUES)

def emis_abatida_kg_existing(m, planta, contaminante):
    if contaminante == 'MP':
        yset = m.y_mp_ex
        paramset = m.param_abatimiento_mp
        equip_set = m.ABATIMIENTO_MP
        col = 'emisiones_mp_kg_mwh'
    elif contaminante == 'SOx':
        yset = m.y_sox_ex
        paramset = m.param_abatimiento_sox
        equip_set = m.ABATIMIENTO_SOX
        col = 'emisiones_sox_kg_mwh'
    else:
        yset = m.y_nox_ex
        paramset = m.param_abatimiento_nox
        equip_set = m.ABATIMIENTO_NOX
        col = 'emisiones_nox_kg_mwh'
    ed = float(m.param_centrales[planta][col])
    return sum(yset[planta, bloque, equipo] * 1000.0 * ed * float(paramset[equipo]['eficiencia']) for bloque in m.BLOQUES for equipo in equip_set)


def emis_reman_kg_existing(m, planta, contaminante):
    return emis_sin_kg_existing(m, planta, contaminante) - emis_abatida_kg_existing(m, planta, contaminante)

model.emis_reman_kg_existing = pyo.Expression(model.CENTRALES, ['MP','SOx','NOx'], rule=emis_reman_kg_existing)

# Repetir para plantas nuevas

def emis_sin_kg_new(m, planta, contaminante):
    if contaminante == 'MP':
        col = 'emisiones_mp_kg_mwh'
    elif contaminante == 'SOx':
        col = 'emisiones_sox_kg_mwh'
    else:
        col = 'emisiones_nox_kg_mwh'
    ed = float(m.param_centrales_nuevas[planta][col])
    return sum(m.generacion_nuevas[planta, bloque] * 1000.0 * ed for bloque in m.BLOQUES)


def emis_abatida_kg_new(m, planta, contaminante):
    if contaminante == 'MP':
        yset = m.y_mp_new
        paramset = m.param_abatimiento_mp
        equip_set = m.ABATIMIENTO_MP
        col = 'emisiones_mp_kg_mwh'
    elif contaminante == 'SOx':
        yset = m.y_sox_new
        paramset = m.param_abatimiento_sox
        equip_set = m.ABATIMIENTO_SOX
        col = 'emisiones_sox_kg_mwh'
    else:
        yset = m.y_nox_new
        paramset = m.param_abatimiento_nox
        equip_set = m.ABATIMIENTO_NOX
        col = 'emisiones_nox_kg_mwh'
    ed = float(m.param_centrales_nuevas[planta][col])
    return sum(yset[planta, bloque, equipo] * 1000.0 * ed * float(paramset[equipo]['eficiencia']) for bloque in m.BLOQUES for equipo in equip_set)


def emis_reman_kg_new(m, planta, contaminante):
    return emis_sin_kg_new(m, planta, contaminante) - emis_abatida_kg_new(m, planta, contaminante)

model.emis_reman_kg_new = pyo.Expression(model.CENTRALES_NUEVAS, ['MP','SOx','NOx'], rule=emis_reman_kg_new)

# Constraint: si la emisión remanente supera la norma (emis_sin*(1-norma)), entonces planta_operable debe ser 0

def cumple_norma_existing_rule(m, planta, contaminante):
    tec = m.param_centrales[planta]['tecnologia']
    if tec in t_centrales_termicas:
        norma = normas_emision.get(tec, {}).get(contaminante, 0.0)
        emis_sin = emis_sin_kg_existing(m, planta, contaminante)
        emis_reman = emis_reman_kg_existing(m, planta, contaminante)
        # emis_reman <= emis_sin*(1-norma) + BigM_emis*(1 - planta_operable)
        return emis_reman <= emis_sin * (1 - norma) + BigM_emis_kg * (1 - m.planta_operable_ex[planta])
    else:
        return pyo.Constraint.Skip

model.cumple_norma_existing = pyo.Constraint(model.CENTRALES, ['MP','SOx','NOx'], rule=cumple_norma_existing_rule)

# Repetir para nuevas

def cumple_norma_new_rule(m, planta, contaminante):
    tec = m.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_centrales_termicas:
        norma = normas_emision.get(tec, {}).get(contaminante, 0.0)
        emis_sin = emis_sin_kg_new(m, planta, contaminante)
        emis_reman = emis_reman_kg_new(m, planta, contaminante)
        return emis_reman <= emis_sin * (1 - norma) + BigM_emis_kg * (1 - m.planta_operable_new[planta])
    else:
        return pyo.Constraint.Skip

model.cumple_norma_new = pyo.Constraint(model.CENTRALES_NUEVAS, ['MP','SOx','NOx'], rule=cumple_norma_new_rule)

# Nota: la lógica activará planta_operable_* = 1 si se cumplen todas las restricciones de los 3 contaminantes; si alguna falla, la planta puede quedar con planta_operable=0 y todas las generaciones se bloquearán.
# Puedes añadir en la función objetivo un término de costo de invertir/operar abatidores si quieres que el solver elija equipos y potrencias de forma económica.
